In [5]:
from pgpy.constants import PubKeyAlgorithm, KeyFlags, HashAlgorithm, SymmetricKeyAlgorithm, CompressionAlgorithm
import pgpy
import os
import glob
from secrets import token_hex

In [6]:
def key_generation(user_name,passphrase,salt):
    key = pgpy.PGPKey.new(PubKeyAlgorithm.RSAEncryptOrSign, 2048)
    uid = pgpy.PGPUID.new(user_name)

    key.add_uid(uid, usage={KeyFlags.Sign, KeyFlags.EncryptCommunications, KeyFlags.EncryptStorage},
            hashes=[HashAlgorithm.SHA256, HashAlgorithm.SHA384, HashAlgorithm.SHA512, HashAlgorithm.SHA224],
            ciphers=[SymmetricKeyAlgorithm.AES256, SymmetricKeyAlgorithm.AES192, SymmetricKeyAlgorithm.AES128],
            compression=[CompressionAlgorithm.ZLIB, CompressionAlgorithm.BZ2, CompressionAlgorithm.ZIP, CompressionAlgorithm.Uncompressed])
    key.protect(passphrase+salt, SymmetricKeyAlgorithm.AES256, HashAlgorithm.SHA256)
    return key

In [7]:
# message is in string given by User
# pub_key is fetched from Database i.e Receivers public Key
def encryption(pub_key,message):
    pub_key_enc_pgpy, _ = pgpy.PGPKey.from_blob(pub_key)
    text_message = pgpy.PGPMessage.new(message)
    encrypted_message = pub_key_enc_pgpy.encrypt(text_message)
    return encrypted_message

In [8]:
# sec_key is Private key stored on local device
# enc_message is encrypted message
def decryption(sec_key,enc_message,passphrase,salt):
    temp_pgpy = pgpy.PGPKey()
    temp_pgpy.parse(sec_key)
    message = pgpy.PGPMessage()
    message.parse(enc_message)
    try:
        with temp_pgpy.unlock(passphrase+salt):
            decrypted_message = temp_pgpy.decrypt(message)
            return decrypted_message.message
    except:
        print("Wrong passphrase")

In [9]:
# sec_key is key stored at local machine i.e private key
# message is string
def sign(sec_key,message,passphrase,salt):
    temp_pgpy = pgpy.PGPKey()
    temp_pgpy.parse(sec_key)
    try:
        with temp_pgpy.unlock(passphrase+salt):
            message = pgpy.PGPMessage.new(message)
            signature = temp_pgpy.sign(message)
            return signature
    except:
        print("Wrong passphrase")

In [10]:
# pub_key is key fetched from DB
# message is signed string message
def verify(pub_key,message,signature):
    pub_key, _ = pgpy.PGPKey.from_blob(pub_key)
    temp_pgpy = pgpy.PGPSignature()
    temp_pgpy.parse(signature)
    if pub_key.verify(message,temp_pgpy):
        return True
    else:
        return False


In [11]:
def get_salt():
    return str(token_hex(16))

In [12]:
def combine(enc,sign):
    return (str(enc)+str(sign))

In [13]:
def separate_enc_sign(combined_msg):
    index_of_sign = combined_msg.find("-----BEGIN PGP SIGNATURE-----")
    extracted_msg = combined_msg[0:index_of_sign]
    extracted_sign = combined_msg[index_of_sign:len(combined_msg)]
    return extracted_msg,extracted_sign

In [25]:
# both keys are in string format
def sign_public_key(public_key,private_key,passphrase,salt):
    public_pgpy = pgpy.PGPKey()
    public_pgpy.parse(public_key)
    private_pgpy = pgpy.PGPKey()
    private_pgpy.parse(private_key)
    try:
        with private_pgpy.unlock(passphrase+salt):
            sign = private_pgpy.certify(public_pgpy)
            return sign
    except:
        print("Wrong passphrase")    

In [15]:
# INFO=======================================
# store the OUTPUT of key_generation function on local machine as private key
# Get public key using key.pubkey method and store it on DB

In [16]:
# Sender Side
salt1 = get_salt()
sender_key = key_generation('sender',"SecretPassphrase1",salt1)
# print(sender_key.pubkey)

In [17]:
#Receiver Side
salt2=get_salt()
receiver_key = key_generation('receiver',"SecretPassphrase2",salt2)
# print(receiver_key.pubkey)

In [18]:
sender_sec_str = str(sender_key) # sec_str is key from local machine
sender_pub_str = str(sender_key.pubkey) # pub_str is key from DB

In [19]:
receiver_sec_str = str(receiver_key) # sec_str is key from local machine
receiver_pub_str = str(receiver_key.pubkey) # pub_str is key from DB

In [28]:
signature_public = sign_public_key(receiver_pub_str,receiver_sec_str,"SecretPassphrase2",salt2)

In [29]:
print(signature_public)

-----BEGIN PGP SIGNATURE-----

wsBfBB8BCAAJBQJeputTAh4BAAoJEKJ8eZVOB14NN5kIAJ5p9m1b9m6yteUbeURq
uVa/p3GzrGFsZ22Sf5y0Iq7pullxIKTR8uKaOH8FzDGjLRul8gCfUmagxEWqKExI
NhCy+P597SGBsnNu5uzPH/wSor1y9TpxrOyJYpIKPVQ60MoDz1vdBE9s0O3+Aj8N
n25lqwXmcRrb2c/wL3LK0MNDIMgicM8CRVk5hzYz7d1HPwJx0lfmYE/Z5lm5gk63
R7rhH1CNH0VfX75GjVgjsfS6zlKZzNWF5XDyYv9OCvMRFYPT1osnqHGjx+S+8hxX
ejHeoKOLkcbaOn2mvtg/+zXQ40NG5mv48Is3rsNZILbtf00jOy06atHoquXu8G3N
0G0=
=dP9w
-----END PGP SIGNATURE-----



In [20]:
#Sender Side
enc = encryption(receiver_pub_str,"This is Time Pass Message!")
# print(enc)
print(type(enc))
# print("encrypted? " + str(enc.is_encrypted))
# print("Signed? " + str(enc.is_signed))

<class 'pgpy.pgp.PGPMessage'>


In [21]:
#Sender Side
signature = sign(sender_sec_str,str(enc),"SecretPassphrase1",salt1)
print(signature)
print(type(signature))

-----BEGIN PGP SIGNATURE-----

wsBcBAABCAAGBQJepuoPAAoJEHs7v9QJ5x2yR/wIAJS00rnF8PzCZekWuUr7KONF
qJvpO2iFAQtkBuHnzV+qQYlap7Ck4m0Onv8cFxhJQUHl0djg3YWzCb3M46Yx3Fol
FvU4fjj3oZBqZzmZPgnZJjrWVttjmJRSk9Zp0o3Hwz4h12NukoqeaoxlyHJA22wF
idYQ7Drf+PMt6O0AxduvFZBTU/pCAYyJS3O4S/lHhUcigl2KADzRDv53fotyuHb8
s5qFG+JeaV/R0OltvZBNwbwbdjeFO0KM1N0VKk4WazgmJ4L/j7LeVBs8fKgvBbeg
bIQGlx72z6XyvAABpbtiJ3uykKZMRk3mhVjVH5TyJljwrDP30wHD+cDCDpSB9kc=
=k0Z1
-----END PGP SIGNATURE-----

<class 'pgpy.pgp.PGPSignature'>


In [22]:
#sender side
combined_msg = combine(enc,signature)
print(combined_msg)

-----BEGIN PGP MESSAGE-----

wcBMA6J8eZVOB14NAQf9EG0jaHkL4PmI4jeHwYMYKGRvi8jErS7TdadQFsWMFlZz
LuIIpH+kp1anJIsXkNVKiU+xrgfYI32ArlvhTSrt2EkM7zua5SuXC5MXFNwHLuKj
fICbFGfpDTjbkXH9+cJV7z9Yax71rEyXkxT7gTv+ch4LID/bVXW9V/tkm3TMgBMv
dk3NQvAJi9clSXkqL1sWljUF+8txbTPwjfwB9Z1sdnbSPdZXOZ/z/hZNhwvH7OOb
Qa2mf02YJMd96Dmg6ScEq7CRDgXUz4b4Ld+Sr6IRFKpRlmkUbqxCbHswQ9yzZBs3
9J3culJgUdziLAgOghO7x94a8QeDlXdsw2YO2uwOOtJPAdhAQw0aHOAXq0XpnPPI
20eggF8khmRCsYEQdsnfCJQxdog8dVwC7xof2N9mA72RgeBX21dmyvhJ4kzh8fVa
jpSpWtIN14kNcNnvQXEaPQ==
=xogS
-----END PGP MESSAGE-----
-----BEGIN PGP SIGNATURE-----

wsBcBAABCAAGBQJepuoPAAoJEHs7v9QJ5x2yR/wIAJS00rnF8PzCZekWuUr7KONF
qJvpO2iFAQtkBuHnzV+qQYlap7Ck4m0Onv8cFxhJQUHl0djg3YWzCb3M46Yx3Fol
FvU4fjj3oZBqZzmZPgnZJjrWVttjmJRSk9Zp0o3Hwz4h12NukoqeaoxlyHJA22wF
idYQ7Drf+PMt6O0AxduvFZBTU/pCAYyJS3O4S/lHhUcigl2KADzRDv53fotyuHb8
s5qFG+JeaV/R0OltvZBNwbwbdjeFO0KM1N0VKk4WazgmJ4L/j7LeVBs8fKgvBbeg
bIQGlx72z6XyvAABpbtiJ3uykKZMRk3mhVjVH5TyJljwrDP30wHD+cDCDpSB9kc=
=k0Z1
-----END PGP SIGNATURE-----



In [33]:
#Receiver side
extracted_msg,extracted_sign = separate_enc_sign(combined_msg)
# print(extracted_msg)

In [34]:
# print(extracted_sign)

In [35]:
#Receiver Side
print(verify(sender_pub_str,str(extracted_msg),str(extracted_sign)))

True


In [36]:
#Receiver Side
print(decryption(receiver_sec_str,extracted_msg,"SecretPassphrase2",salt2))

This is Time Pass Message!
